Josh Hoge

jsh10043@nyu.edu

NYU Tandon Data Science Bootcamp Week 5 Homework

In [14]:
"""
https://leetcode.com/problems/actors-and-directors-who-cooperated-at-least-three-times/

SQL solution:

SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;
"""

import pandas as pd

data = [[1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 3], [1, 2, 4], [2, 1, 5], [2, 1, 6]]
actor_director = pd.DataFrame(data, columns=['actor_id', 'director_id', 'timestamp']).astype({'actor_id':'int64', 'director_id':'int64', 'timestamp':'int64'})

df = pd.DataFrame(actor_director)

# group by actor_id and director_id
df_grouped = df.groupby(["actor_id", "director_id"]).size()

# create a mask to select only pairs with count >=3
mask_series = df_grouped[df_grouped >= 3]

# convert the index of mask_series into a DataFrame with columns 'actor_id' and 'director_id'
result_df = pd.DataFrame(mask_series.index.tolist(), columns=["actor_id", "director_id"])

print(result_df)

   actor_id  director_id
0         1            1


In [13]:
"""
https://leetcode.com/problems/fix-names-in-a-table/

SQL solution:

SELECT
    user_id,
    CONCAT(UPPER(SUBSTR(name, 1, 1)), LOWER(SUBSTR(name, 2))) as name
FROM
    Users
ORDER BY
    user_id;
"""

import pandas as pd

data = [[1, 'aLice'], [2, 'bOB']]
users = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id':'Int64', 'name':'object'})

df = pd.DataFrame(users)

# use the title() function to normalize the letter cases
users['name'] = users['name'].str.title()

print(users)

   user_id   name
0        1  Alice
1        2    Bob


In [15]:
"""
https://leetcode.com/problems/combine-two-tables/

SQL solution:

SELECT
    p.firstName,
    p.lastName,
    a.city,
    a.state
FROM
    Person p
LEFT JOIN
    Address a ON p.personId = a.personId;
"""
import pandas as pd

data = [[1, 'Wang', 'Allen'], [2, 'Alice', 'Bob']]
person = pd.DataFrame(data, columns=['personId', 'firstName', 'lastName']).astype({'personId':'Int64', 'firstName':'object', 'lastName':'object'})

data = [[1, 2, 'New York City', 'New York'], [2, 3, 'Leetcode', 'California']]
address = pd.DataFrame(data, columns=['addressId', 'personId', 'city', 'state']).astype({'addressId':'Int64', 'personId':'Int64', 'city':'object', 'state':'object'})

# this is the left join
result = pd.merge(person, address, on='personId', how='left')

# select the desired columns
result = result[['firstName', 'lastName', 'city', 'state']]

print(result)

  firstName lastName           city     state
0      Wang    Allen            NaN       NaN
1     Alice      Bob  New York City  New York


In [24]:
"""
https://leetcode.com/problems/second-highest-salary/

SQL solution:

SELECT MAX(salary) AS SecondHighestSalary
FROM Employee
WHERE salary < (SELECT DISTINCT MAX(salary) FROM Employee);
"""

import pandas as pd

data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})

# get the distinct salaries and sort in descending order
distinct_salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)

# check if we have at least two distinct salaries
if len(distinct_salaries) >= 2:
    second_highest = distinct_salaries.iloc[1]   # grab the 2nd element
else:
    second_highest = None

print("Second highest salary:", second_highest)


Second highest salary: None


In [30]:
"""
https://leetcode.com/problems/list-the-products-ordered-in-a-period/

SQL solution:

SELECT p.product_name, SUM(o.unit) AS unit
FROM Products p
JOIN Orders o ON p.product_id = o.product_id
WHERE MONTH(o.order_date) = 2 AND YEAR(o.order_date) = 2020
GROUP BY p.product_id, p.product_name
HAVING SUM(o.unit) >= 100;
"""

import pandas as pd

data_products = [
    [1, 'Leetcode Solutions',  'Book'],
    [2, 'Jewels of Stringology','Book'],
    [3, 'HP',                   'Laptop'],
    [4, 'Lenovo',                'Laptop'],
    [5, 'Leetcode Kit',          'T‑shirt']
]

products = pd.DataFrame(
    data_products,
    columns=['product_id', 'product_name', 'product_category']
).astype({'product_id':'Int64',
           'product_name':'object',
           'product_category':'object'})

data_orders = [
    [1, '2020-02-05', 60],
    [1, '2020-02-10', 70],
    [2, '2020-01-18', 30],
    [2, '2020-02-11', 80],
    [3, '2020-02-17', 2],
    [3, '2020-02-24', 3],
    [4, '2020-03-01', 20],
    [4, '2020-03-04', 30],
    [4, '2020-03-04', 60],
    [5, '2020-02-25', 50],
    [5, '2020-02-27', 50],
    [5, '2020-03-01', 50]
]

orders = pd.DataFrame(
    data_orders,
    columns=['product_id', 'order_date', 'unit']
).astype({'product_id':'Int64',
           'order_date':'datetime64[ns]',
           'unit':'Int64'})

# filter orders from FEB
feb_2020 = orders[
    (orders['order_date'].dt.year == 2020) &
    (orders['order_date'].dt.month == 2)
]

# aggregate units per product_id
feb_agg = (
    feb_2020
    .groupby('product_id', as_index=False)['unit']
    .sum()
    .rename(columns={'unit': 'amount'})
)

# discard products with less than 100 units
feb_agg = feb_agg[feb_agg['amount'] >= 100]

# natural join to get the product name
result = feb_agg.merge(products[['product_id', 'product_name']],
                       on='product_id',
                       how='left')

print(result)


   product_id  amount        product_name
0           1     130  Leetcode Solutions
1           5     100        Leetcode Kit


In [33]:
"""
https://leetcode.com/problems/replace-employee-id-with-the-unique-identifier/

SQL solution:

SELECT
        eu.unique_id,
        e.name
FROM Employees AS e
LEFT JOIN EmployeeUNI AS eu
      ON e.id = eu.id
ORDER BY

        CASE WHEN eu.unique_id IS NULL THEN 0 ELSE 1 END,
        e.id;
"""

import pandas as pd

data = [[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']]
employees = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'int64', 'name':'object'})
data = [[3, 1], [11, 2], [90, 3]]
employee_uni = pd.DataFrame(data, columns=['id', 'unique_id']).astype({'id':'int64', 'unique_id':'int64'})

import pandas as pd

# build the tables
employees = pd.DataFrame(
    [[1, 'Alice'],
     [7, 'Bob'],
     [11, 'Meir'],
     [90, 'Winston'],
     [3, 'Jonathan']],
    columns=['id', 'name']
).astype({'id': 'int64', 'name': 'object'})

employee_uni = pd.DataFrame(
    [[3, 1],
     [11, 2],
     [90, 3]],
    columns=['id', 'unique_id']
).astype({'id': 'int64', 'unique_id': 'int64'})

# left join on employee.id = employee_uni.id
merged = employees.merge(employee_uni, how='left', on='id')

# pick the columns we want to print
result = merged[['unique_id', 'name']].sort_values(
    by='unique_id',
    na_position='first'
)

print(result)


   unique_id      name
0        NaN     Alice
1        NaN       Bob
4        1.0  Jonathan
2        2.0      Meir
3        3.0   Winston


In [36]:
"""
https://leetcode.com/problems/game-play-analysis-iv/

SQL solution: (MySQL specific, using a CTE)

WITH first_logins AS (
    SELECT
        player_id,
        MIN(event_date)  AS first_login   -- the day a player logged in for the first time
    FROM Activity
    GROUP BY player_id
)

SELECT ROUND(
           COUNT(DISTINCT a.player_id) * 1.0 /
           (SELECT COUNT(*) FROM first_logins),
           2)
       AS fraction_of_players_back_next_day
FROM first_logins f
JOIN Activity a
      ON  a.player_id = f.player_id
      AND a.event_date = DATE_ADD(f.first_login, INTERVAL 1 DAY);
"""

import pandas as pd

data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})


# find first login dates
first_login = (
    activity.groupby('player_id')['event_date']
            .min()
            .reset_index(name='first_login')
)

# merge first login info back into the first table
activity_with_first = activity.merge(first_login, on='player_id', how='inner')

# keep rows that are exactly one day after the first login
next_day = activity_with_first[
    activity_with_first['event_date'] ==
    activity_with_first['first_login'] + pd.Timedelta(days=1)
]

# compute the fraction of players that logged in on the next day
players_with_next_day = next_day['player_id'].unique()
num_players_with_next_day = len(players_with_next_day)

total_unique_players = activity['player_id'].nunique()

fraction = num_players_with_next_day / total_unique_players
rounded_fraction = round(fraction, 2)

print("Fraction of players back the next day:", rounded_fraction)


Fraction of players back the next day: 0.33


In [38]:
"""
https://leetcode.com/problems/project-employees-i/

SQL solution:

SELECT
    p.project_id,
    ROUND(AVG(e.experience_years), 2) AS average_years
FROM Project p
JOIN Employee e ON p.employee_id = e.employee_id
GROUP BY p.project_id;
"""


import pandas as pd

data = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id']).astype({'project_id':'Int64', 'employee_id':'Int64'})
data = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 1], [4, 'Doe', 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'name', 'experience_years']).astype({'employee_id':'Int64', 'name':'object', 'experience_years':'Int64'})

# join the tables
merged = project.merge(employee, on='employee_id', how='inner')

# group-by and compute average
result = (
    merged
    .groupby('project_id', as_index=False)
    .agg(avg_experience=('experience_years', lambda x: round(x.mean(), 2)))
)

print(result)


   project_id  avg_experience
0           1             2.0
1           2             2.5


In [40]:
"""
https://leetcode.com/problems/department-top-three-salaries/

SQL solution:

WITH dept_salaries AS (
    SELECT DISTINCT departmentId, salary
    FROM Employee
),
ranked_salaries AS (
    SELECT
        departmentId,
        salary,
        DENSE_RANK() OVER (PARTITION BY departmentId ORDER BY salary DESC) AS rk
    FROM dept_salaries
)

SELECT
    d.name          AS Department,
    e.name          AS Employee,
    e.salary        AS Salary
FROM Employee       AS e
JOIN Department     AS d   ON e.departmentId = d.id
JOIN ranked_salaries AS r   ON e.departmentId = r.departmentId
                           AND e.salary      = r.salary
WHERE r.rk <= 3
ORDER BY d.name, e.salary DESC;
"""

import pandas as pd

data = [[1, 'Joe', 85000, 1], [2, 'Henry', 80000, 2], [3, 'Sam', 60000, 2], [4, 'Max', 90000, 1], [5, 'Janet', 69000, 1], [6, 'Randy', 85000, 1], [7, 'Will', 70000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

# rank each salary with department, using dense ranking
employee['salary_rank'] = (
    employee
    .groupby('departmentId')['salary']
    .rank(method='dense', ascending=False)
)

# keep the highest 3 salaries in each department
high_earners = employee[employee['salary_rank'] <= 3].copy()

# merge with department table to get department name
high_earners = high_earners.merge(
    department,
    left_on='departmentId',
    right_on='id',
    how='left'
)

# re-order the columns so that they match the desired table output
result = (
    high_earners[['name_y', 'name_x', 'salary']]
    .rename(columns={'name_y': 'Department',
                     'name_x': 'Employee'})
    .sort_values(['Department', 'salary'],
                 ascending=[True, False])
    .reset_index(drop=True)
)

print(result)


  Department Employee  salary
0         IT      Max   90000
1         IT      Joe   85000
2         IT    Randy   85000
3         IT     Will   70000
4      Sales    Henry   80000
5      Sales      Sam   60000
